<a href="https://colab.research.google.com/github/EuclidStellar/Model-Bechmarks-for-stt/blob/main/DataextractionTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Hugging Face's transformers library
!pip install transformers
!pip install torch

# Import necessary libraries
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import pipeline

# Load the pre-trained model and tokenizer from Hugging Face
model_name = "mdarhri00/named-entity-recognition"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a pipeline for Named Entity Recognition (NER)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Example voice input
voice_input = "Hi my name is Gaurav Singh, I was born on 25 April 2005 and I want to choose banana from the dropdown."

# Perform named entity recognition (NER) on the voice input
ner_results = nlp(voice_input)

# Display the NER results
print("NER Results:", ner_results)

# Parse the NER results to extract relevant information
def extract_entities(ner_results):
    name = ""
    date_of_birth = ""
    dropdown_selection = ""

    for entity in ner_results:
        if entity['entity'] == 'PER':  # Assuming PER is used for Person's name
            name += entity['word'] + " "
        elif entity['entity'] == 'DATE':  # DATE entity
            date_of_birth += entity['word'] + " "
        elif entity['entity'] == 'MISC':  # Assuming MISC is for dropdown selection
            dropdown_selection += entity['word'] + " "

    return {
        "name": name.strip(),
        "date_of_birth": date_of_birth.strip(),
        "dropdown_selection": dropdown_selection.strip()
    }

# Extracted data
extracted_data = extract_entities(ner_results)

# Display the extracted data
print("\nExtracted Data:")
print(f"Name: {extracted_data['name']}")
print(f"Date of Birth: {extracted_data['date_of_birth']}")
print(f"Dropdown Selection: {extracted_data['dropdown_selection']}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/709M [00:00<?, ?B/s]

NER Results: [{'entity': 'Person_Name', 'score': 0.99307185, 'index': 5, 'word': 'Gaur', 'start': 14, 'end': 18}, {'entity': 'Person_Name', 'score': 0.99302363, 'index': 6, 'word': '##av', 'start': 18, 'end': 20}, {'entity': 'Person_Name', 'score': 0.99183613, 'index': 7, 'word': 'Singh', 'start': 21, 'end': 26}, {'entity': 'date_time', 'score': 0.9959936, 'index': 13, 'word': '25', 'start': 42, 'end': 44}, {'entity': 'date_time', 'score': 0.99763083, 'index': 14, 'word': 'April', 'start': 45, 'end': 50}, {'entity': 'date_time', 'score': 0.99692696, 'index': 15, 'word': '2005', 'start': 51, 'end': 55}]

Extracted Data:
Name: 
Date of Birth: 
Dropdown Selection: 


In [2]:
# Function to properly extract and handle subword tokens (tokens starting with ##)
def extract_entities(ner_results):
    name = ""
    date_of_birth = ""
    dropdown_selection = ""

    current_entity = None
    current_text = ""

    for entity in ner_results:
        word = entity['word']

        # If the word is a subword (starts with ##), concatenate it with the previous part of the word
        if word.startswith('##'):
            current_text += word[2:]  # Remove '##' and concatenate
        else:
            # If we've reached a new entity, save the previous entity text
            if current_entity and current_text:
                if current_entity == 'Person_Name':
                    name += current_text + " "
                elif current_entity == 'date_time':
                    date_of_birth += current_text + " "
                elif current_entity == 'dropdown':
                    dropdown_selection += current_text + " "

            # Start a new entity
            current_entity = entity['entity']
            current_text = word

    # Add the last entity text
    if current_entity and current_text:
        if current_entity == 'Person_Name':
            name += current_text + " "
        elif current_entity == 'date_time':
            date_of_birth += current_text + " "
        elif current_entity == 'dropdown':
            dropdown_selection += current_text + " "

    return {
        "name": name.strip(),
        "date_of_birth": date_of_birth.strip(),
        "dropdown_selection": dropdown_selection.strip()
    }

# Extracted data
extracted_data = extract_entities(ner_results)

# Display the extracted data
print("\nExtracted Data:")
print(f"Name: {extracted_data['name']}")
print(f"Date of Birth: {extracted_data['date_of_birth']}")
print(f"Dropdown Selection: {extracted_data['dropdown_selection']}")



Extracted Data:
Name: Gaurav Singh
Date of Birth: 25 April 2005
Dropdown Selection: 


In [3]:
# Import necessary libraries
import re

# Function to properly extract and handle subword tokens (tokens starting with ##)
def extract_entities(ner_results, voice_input):
    name = ""
    date_of_birth = ""
    dropdown_selection = ""

    current_entity = None
    current_text = ""

    # Iterate through the NER results
    for entity in ner_results:
        word = entity['word']

        # Handle subwords (tokens starting with ##)
        if word.startswith('##'):
            current_text += word[2:]  # Remove '##' and concatenate
        else:
            # If we've reached a new entity, save the previous entity text
            if current_entity and current_text:
                if current_entity == 'Person_Name':
                    name += current_text + " "
                elif current_entity == 'date_time':
                    date_of_birth += current_text + " "

            # Start a new entity
            current_entity = entity['entity']
            current_text = word

    # Add the last entity text
    if current_entity and current_text:
        if current_entity == 'Person_Name':
            name += current_text + " "
        elif current_entity == 'date_time':
            date_of_birth += current_text + " "

    # Manual extraction for dropdown selection using a regex pattern
    dropdown_match = re.search(r"I want to choose (\w+)", voice_input)
    if dropdown_match:
        dropdown_selection = dropdown_match.group(1)

    return {
        "name": name.strip(),
        "date_of_birth": date_of_birth.strip(),
        "dropdown_selection": dropdown_selection.strip()
    }

# Example voice input
voice_input = "Hi my name is Gaurav Singh, I was born on 25 April 2005 and I want to choose banana from the dropdown."

# Perform named entity recognition (NER) on the voice input
ner_results = nlp(voice_input)

# Extracted data
extracted_data = extract_entities(ner_results, voice_input)

# Display the extracted data
print("\nExtracted Data:")
print(f"Name: {extracted_data['name']}")
print(f"Date of Birth: {extracted_data['date_of_birth']}")
print(f"Dropdown Selection: {extracted_data['dropdown_selection']}")



Extracted Data:
Name: Gaurav Singh
Date of Birth: 25 April 2005
Dropdown Selection: banana


In [6]:
# Import necessary libraries
import re

# Function to properly extract and handle subword tokens (tokens starting with ##)
def extract_entities(ner_results, voice_input):
    name = ""
    date_of_birth = ""
    dropdown_selection = ""

    current_entity = None
    current_text = ""

    # Iterate through the NER results
    for entity in ner_results:
        word = entity['word']

        # Handle subwords (tokens starting with ##)
        if word.startswith('##'):
            current_text += word[2:]  # Remove '##' and concatenate
        else:
            # If we've reached a new entity, save the previous entity text
            if current_entity and current_text:
                if current_entity == 'Person_Name':
                    name += current_text + " "
                elif current_entity == 'date_time':
                    date_of_birth += current_text + " "

            # Start a new entity
            current_entity = entity['entity']
            current_text = word

    # Add the last entity text
    if current_entity and current_text:
        if current_entity == 'Person_Name':
            name += current_text + " "
        elif current_entity == 'date_time':
            date_of_birth += current_text + " "

    # Manual extraction for dropdown selection using a regex pattern
    dropdown_match = re.search(r"I want to choose (\w+)", voice_input)
    if dropdown_match:
        dropdown_selection = dropdown_match.group(1)

    return {
        "name": name.strip(),
        "date_of_birth": date_of_birth.strip(),
        "dropdown_selection": dropdown_selection.strip()
    }

# Example voice input
voice_input = "name Gaurav Singh DOB 25 April 2005 and Gender Male"

# Perform named entity recognition (NER) on the voice input
ner_results = nlp(voice_input)

# Extracted data
extracted_data = extract_entities(ner_results, voice_input)

# Display the extracted data
print("\nExtracted Data:")
print(f"Name: {extracted_data['name']}")
print(f"Date of Birth: {extracted_data['date_of_birth']}")
print(f"Dropdown Selection: {extracted_data['dropdown_selection']}")



Extracted Data:
Name: Gaurav Singh
Date of Birth: 25 April 2005
Dropdown Selection: 


In [7]:
import re

# Function to extract entities and handle dropdowns with predefined options
def extract_entities(ner_results, voice_input):
    name = ""
    date_of_birth = ""
    dropdown_selection = ""

    current_entity = None
    current_text = ""

    # List of predefined dropdown options (Gender in this case)
    gender_options = ["Male", "Female"]

    # Iterate through the NER results
    for entity in ner_results:
        word = entity['word']

        # Handle subwords (tokens starting with ##)
        if word.startswith('##'):
            current_text += word[2:]  # Remove '##' and concatenate
        else:
            # If we've reached a new entity, save the previous entity text
            if current_entity and current_text:
                if current_entity == 'Person_Name':
                    name += current_text + " "
                elif current_entity == 'date_time':
                    date_of_birth += current_text + " "

            # Start a new entity
            current_entity = entity['entity']
            current_text = word

    # Add the last entity text
    if current_entity and current_text:
        if current_entity == 'Person_Name':
            name += current_text + " "
        elif current_entity == 'date_time':
            date_of_birth += current_text + " "

    # Manual extraction for gender dropdown by matching predefined options
    for option in gender_options:
        if option.lower() in voice_input.lower():
            dropdown_selection = option
            break

    return {
        "name": name.strip(),
        "date_of_birth": date_of_birth.strip(),
        "dropdown_selection": dropdown_selection.strip()
    }

# Example voice input
voice_input = "Name Gaurav Singh DOB 25 April 2005 and Gender Male"

# Perform named entity recognition (NER) on the voice input
ner_results = nlp(voice_input)

# Extracted data
extracted_data = extract_entities(ner_results, voice_input)

# Display the extracted data
print("\nExtracted Data:")
print(f"Name: {extracted_data['name']}")
print(f"Date of Birth: {extracted_data['date_of_birth']}")
print(f"Dropdown Selection: {extracted_data['dropdown_selection']}")



Extracted Data:
Name: Gaurav Singh
Date of Birth: 25 April 2005
Dropdown Selection: Male
